# I - Script de la partie "Récupération des données". 

# I.1 -  Récupération massive du Registre Nationale des Entreprises de l'Inpi :


Ce NoteBook constitue la première partie du processus de récupération des données. Pour récupérer les données d'informations et d'identification des entreprises francaises, deux choix en open data s'offrent à nous : 
- $i$) `la base Sirene proposée et maintenue par l'Insee` 
- $ii$) `la base du registre nationale des entreprises (RNE) proposé en accès libre et maintenue par l'Inpi`

Nous avons besoin d'informations concernant la possible radiation ou non d'une entreprise donné, de la nature de l'activité de l'établissement principal, des établissements secondaires, de l'âge du/des dirigeant, de leurs expériences en entreprenariat, etc. 
La base Sirene, bien que complète du point de vue d'informations primaires, telles la cessation ou la nature des activités, n'est pas adaptée à notre problème.
A l'inverse, la base RNE est idéale : elle contient bien plus d'information que la base Sirene. 

Nous allons donc récupérer la base RNE. Nous voulons récupérer dans un premier temps, de manière massive, l'ensemble des entreprises créees dans les secteurs du commerce, artisanat et artisanat réglementé, et cela pour la période couvrant 1980 - 2024.

### Deux moyens sont proposés par l'Inpi pour récupérer cette base : 

- Le premier est la fourniture d'une `API` simple, dont vous trouverez une documentation complète en cliquant sur le lien 
suivant : https://www.inpi.fr/sites/default/files/2025-06/documentation%20technique%20API%20formalit%C3%A9s_v4.0.pdf.
Les mises à jours sont quotidiennes.
Un appel par le biais de cette API donne un ou plusieurs résultats au format json. 
La description détaillé de l'arboressance des json est détaillé ici : https://www.inpi.fr/sites/default/files/2025-06/Dictionnaire_de_donnees_INPI_2025_05_09.xlsx.
Ce moyen est parfait pour récupérer des données de manière ponctuelle. Elle n'est donc pas vraiment adaptée à notre problème, bien qu'elle pourra servir à l'avenir.
En effet, elle limite l'accès à 10 000 résultats par pages, avec des requêtes ne pouvant répondre plus de 100 résultats. Ce qui limite grandement les marches de manoeuvres pour 
une récupération massive, sachant que la création d'entreprise n'a fait que de croître ces dernières années.

- Le second est la fourniture d'un accès ouvert au serveur `FTP` (File Transfer Protocol) mis à disposition par l'Inpi. Celui-çi donne accès à la base complète des formalités RNE et est mis à jour sur une fréquence qui semble mensuelle (cette information n'est pas disponible explicitement sur le site de l'Inpi, mais est présente dans les informations du fichier RNE du serveur ,voire plus loin). L'avantage de cette méthode est qu'elle permet de récupérer massivement l'ensemble de la base en téléchargeant simplement celle-çi. Cependant, la volumétrie des données est énorme : plus de 130 gb une fois le fichier (compressé) décompressé. Il faudra donc prendre cela en compte dans la récupération.
Vous trouverez une documentation à l'adresse suivante : https://data.inpi.fr/content/editorial/Serveur_ftp_entreprises.

Nous choisissons donc la deuxième méthode pour une récupération massive.

Dans tous les cas, bien que ouverts, `les accès nécessitent la création d'un compte Inpi` à l'adresse suivante : 
https://data.inpi.fr/register.


## I.1.1 - Stratégie de récupération et de stockage
Par simplicité et reproductibilité, nous téléchargeons directement la base RNE depuis le serveur FTP mis à disposition par l'Inpi directement vers un dossier "diffusion" public du ssp cloud S3. Puis nous traitons en streaming la récupération de l'arborescence de chaque fichier json présent dans le fichier zip téléchargé, et nous importons 13 fichiers aux formats parquet directement dans un dossier "data_RNE", lui même sous-dossier de "diffusion", et donc accessible publiquement.

La stratégie est la suivante :

- i) Nous téléchargeons d'abord la base de donnée grâce à la librairie ftplib permettant de communiquer avec le serveur FTP de l'Inpi. 
Attention : cette partie nécessitera la récupération de votre `Token` dédié sur le site de l'Inpi ainsi que vos identifiants Inpi. 
Nous utilisons la librairie getpass pour la sécurité du token et de l'identifiant. Le fichier obtenu est un fichier au format `zip`, stocké dans le ssp cloud S3.
Le fichier originel compte plus de 10 000 json, eux-mêmes contenants de nombreuses formalités. La fonction  `getRNEfrom_FTP` permet cela de récupérer ces formalités.

- ii) Une fois cette base récupérée, nous implémentons une fonction nommée `get_RNE_past` permettant une lecture de chaque fichier json et ainsi sélectionner les entreprises selon des critères de dates et de départements. Par défaut, nous récupérons les entreprises pour les secteurs suivants : ARTISANALE, ARTISANALE REGLEMENTEE , COMMERCIALE, sur lesquelles nous nous concentrerons.
Nous utilisons la librairie zipfile pour cela, qui permet une lecture de chaque fichier sans décompresser le zip directement.
Nous utilisons `yield` natif de python afin de ne pas utiliser de listes. `yield` permet de ne pas saturer la mémoire vive puisqu'au lieu de retourner une énorme liste de dictionnaire dans notre cas, cette fonctionnalité permet d'itérer directement sur la fonction elle-même, renvoyant ainsi pas à pas chaque dictionnaire.

- iii) Il faut remonter l'arborescence de chaque dictionnaire. La fonction `get_RNE_DataFrame` permet cela. Pour une documentation et une visualisation complète de cette arborescence, nous vous renvoyons ici :https://www.inpi.fr/sites/default/files/2025-06/Dictionnaire_de_donnees_INPI_2025_05_09.xlsx. Nous obtenons ainsi par la suite, de manière assez laborieuse, 13 fichiers parquet. Les fichiers sont enregistrés directement sur le ssp cloud S3. 
Nous précisons ici que nous récupérons le maximum de variables possibles, tout en restant cohérent avec la visée de notre projet. Il ne s'agit pas ici d'une partie de traitement et de selection des variables.





In [1]:
#Packages utilisés
import json
import pandas as pd
import zipfile
from tqdm import tqdm
import datetime
from ftplib import FTP
import getpass
import os
import s3fs
import numpy as np
import duckdb

In [ ]:
#Connexion à S3
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass()
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass()
os.environ["AWS_SESSION_TOKEN"] = getpass.getpass()
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

## i) Téléchargement de la base RNE depuis le serveur FTP de l'Inpi

Comme pour l'utilisation de l'API RNE, il suffit de se créer un compte INPI et de demander une souscription à l'utilisation du serveur ftp RNE. Les identifiants sont données directement sur le site de l'INPI une fois connecté, dans l'espace Mon Compte puis mes services API. On utilise les librairies ftplib et zipfile natives de python pour récupérer les données. Il est à noté que les mises à jours ne sont pas quotidiennes contrairement à l'API RNE mais mensuelles.

Doc : https://www.inpi.fr/sites/default/files/2025-06/documentation%20technique%20API_comptes_annuels%20v5.pdf

In [ ]:
#mdp/id Inpi ftp à rentrer 
username = getpass.getpass()
password = getpass.getpass()

In [ ]:
def getRNEfrom_FTP(username : str, password : str, server_adress : str = "www.inpi.net") -> None:
    
     """
    Fonction permettant la connexion au serveur FTP de l'Inpi, affichant les fichiers disponibles, puis
    téléchargeant le fichier des formalités RNE directement sur le ssp cloud S3.

    Changer le nom du Bucket "MY_BUCKET" si besoin, ainsi que FILE_PATH_OUT_S3.

     inputs : 
               - username : identifiant inpi
               - password : mot de passe inpi
               - server_adress : valeur par défaut = www.inpi.net
     
     outputs :
               ne retourne pas d'objet
     
     """
     #Connexion au serveur FTP
     ftp = FTP(server_adress)
     ftp.login(user=username, passwd=password)

     print("Connexion réussie à :", server_adress)
     print("Répertoire courant :", ftp.pwd())
     
     #Pour afficher le répertoire
     file_list = []
     print(ftp.retrlines('LIST'))
     
     #Récupération du nom du fichier
     ftp.retrlines('LIST',file_list.append)
     file = file_list[-1][59:]
     

     #Upload du fichier zip sur S3 dans diffusion
     MY_BUCKET = "guillaume176"
     FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/{file}"

     print(f"Téléchargement de {file} et stockage dans S3")
     
     #Téléchargement direct depuis le serveur ftp vers S3
     with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
          ftp.retrbinary(f"RETR {file}", file_out.write)
     

     print(f"Fichier '{file}' téléchargé avec succès")

     ftp.quit()


In [21]:
getRNEfrom_FTP(username=username, password=password)

Connexion réussie à : www.inpi.net
Répertoire courant : /
-rw-r--r--    1 ftp      ftp      4078344770 Oct 21 14:14 stock_RNE_comptes_annuels_20250926_1000.zip
-rw-r--r--    1 ftp      ftp      13708395381 Oct 13 10:09 stock_RNE_formalites_20250523_0000.zip
226 Directory send OK.
Téléchargement de stock_RNE_formalites_20250523_0000.zip et stockage dans S3
Fichier 'stock_RNE_formalites_20250523_0000.zip' téléchargé avec succès


On vérifie que le fichier zip s'est bien télécharger dans le fichier diffusion (public) du ssp cloud S3 : 

In [ ]:
path_diffusion = "guillaume176/diffusion"
print(fs.ls(path_diffusion))

['guillaume176/diffusion/.keep', 'guillaume176/diffusion/data_RNE', 'guillaume176/diffusion/stock_RNE_formalites_20250523_0000.zip']


## ii) Récupération du json pour les entreprises des secteurs commerciales, artisanales et artisanales réglementée, créées entre 1980 et 2024.

In [ ]:

def get_RNE_past(date_mini : list  ,date_maxi : list) -> list:

    """
    Fonction permettant la lecture des fichiers json depuis le cloud ssp cloud S3 en selectionnant 
    intervalle de dates. Récupère les entreprises crées dans cet intervalle de dates pour l'ensemble du territoire Français.

    Changer la liste act selon les besoins.

     inputs : 
               - date_mini : borne inf incluse intervalle de dates type [year,month,day] = [2024,1,1]
               - date_maxi : brone sup non-incluse intervalle de dates type [year,month,day] = [2025,8,2]
     
     outputs :
               ne retourne pas d'objet explicite, mais chaque dictionnaire portant sur une firme 
               pas à pas grâce à la fonctionnalité yields
     
     """
    
    #Liste des secteurs d'activités sur lesquels travailler
    act = ["ARTISANALE","ARTISANALE_REGLEMENTEE","COMMERCIALE"]
    
    date_min = datetime.date(date_mini[0],date_mini[1],date_mini[2])
    date_max = datetime.date(date_maxi[0],date_maxi[1],date_maxi[2])
    
 
    #Récupération du chemin du fichier zip enregistré précédemment depuis diffusion sur le ssp cloud S3
    path = fs.ls("guillaume176/diffusion")[-1]

    
    with fs.open(f"s3://{path}","rb") as s3_file:
        with zipfile.ZipFile(s3_file) as archive:
            
            for name in tqdm(archive.namelist()):
                
                with archive.open(name,mode="r") as f:
                    
                    json_it = f.read().decode('utf-8')
                json_it = json_it[:-1] + "]"  #Corrige ici un bug dû à une erreur de formatage native des fichiers json
                json_it = json.loads(json_it)
                
                for firme in json_it:
                    
                    if firme.get("formality").get("content").get("natureCreation").get("dateCreation") is not None:
                        
                        dateCreation = firme.get("formality").get("content").get("natureCreation").get("dateCreation")
                        dateCreation = datetime.datetime.strptime(dateCreation, "%Y-%m-%d").date()
                        
                        activitePrincipale = firme.get("formality").get("content").get("formeExerciceActivitePrincipale","non-disp")
                        
                        if activitePrincipale in act:
                            if (dateCreation >= date_min) and (dateCreation < date_max):
                                personneMorale = firme.get("formality").get("content").get("personneMorale")
                                personnePhysique = firme.get("formality").get("content").get("personnePhysique")
                                
                                if (personneMorale is not None) or (personnePhysique is not None):
                                    yield firme
                                else:
                                    continue
                                        
                                

## iii) Récupération des informations utiles en parcourant l'arborescence de chaque json

In [ ]:
def prepare_RNE_Dict() -> dict:
    
    """
    Fonction permettant la création du dictionnaire de travail pour la récupération des données et la mise sous forme
    de data frame.

    Changer variables_rne si besoins. Changer autre_... et associe_... si besoins.

     inputs : 
               None
     
     outputs :
               RNE : dictionnaire dont les clefs sont les variables retenues, à modifier/ajouter selon les besoins.
               Voire documentation Inpi pour les variables présentes dans la base du RNE.
     
     """
    
    RNE = {}

    variables_rne = [
        "siren", "cessation", "date_creation", "etranger", "micro", "agricole", "eirl",
        "code_postal", "codeInseeCommune", "voie", "typeVoie", "numVoie",
        "dateRadiation", "dateEffet", "denomination", "formeJuridique", "codeAPE",
        "dateImmat", "dateDebutAct", "personneMorale", "nicSiege",
        "objet", "capitalVariable", "montantCapital", "formeExerciceActivitePrincipale",
        "nom", "prenoms", "genre", "rolePourEntreprisePrincipal","statutPourFormalitePrincipal",
        "principalAPEPrincipal","principalDateTransPrincipal","indicateurDecesEntrepreneur","dateCessationTotaleActivite",
        "dateClotureLiquidation","dateDissolutionDisparition","indicateurCessationTemporaire","indicateurPoursuiteActivite",
        "indicateurMaintienImmatriculationRegistre","indicateurDissolution","indicateurDisparitionPM","motifDisparition",
        "dateMiseEnSommeil","typeDissolution","nb_autres","diffusionINSEE","diffusionCommerciale","age_physique"
        
                    ]


    autre_ape = [f"autre_{i}_ape" for i in range(1,11)]
    autre_statutFormalites = [f"autre_{i}_statutFormalites" for i in range(1,11)]
    autre_role = [f"autre_{i}_role" for i in range(1,11)]
    autre_adresse = [f"autre_{i}_adresse" for i in range(1,11)]
    autre_datefin = [f"autre_{i}_datefin" for i in range(1,11)]
    autre_dateeffetferm = [f"autre_{i}_dateeffetferm" for i in range(1,11)]
    autre_datedebut = [f"autre_{i}_datedebut" for i in range(1,11)]
    autre_dateeffettrans = [f"autre_{i}_dateeffettrans" for i in range(1,11)]
    
    associe_role = [f"associe_{i}_role" for i in range(1,6)]
    associe_nom = [f"associe_{i}_nom" for i in range(1,6)]
    associe_prenom = [f"associe_{i}_prenom" for i in range(1,6)]
    associe_naissance = [f"associe_{i}_naissance" for i in range(1,6)]
    associe_id = [f"associe_{i}_id" for i in range(1,6)]
    
    variables_rne.extend([*autre_ape,*autre_statutFormalites,*autre_role,*autre_adresse,*autre_datefin,*autre_dateeffetferm,
                        *autre_datedebut,*autre_dateeffettrans,*associe_role,*associe_prenom,*associe_nom,*associe_id,*associe_naissance])
    
    for var in variables_rne:
        RNE[var] = []  

    return RNE

In [ ]:

def get_RNE_DataFrame(firme : dict) -> dict:

    """
    Fonction permettant le parcours des arborescences d'un dictionnaire donné d'une firme donnée 
    dont la nomenclature est celle du RNE Inpi.

    A adapter selon les cas en lien avec la fonction prepare_RNE_Dict().

     inputs : 
               firme : dictionnaire issu d'un json RNE Inpi contenant l'aborescence d'une firme donnée
     
     outputs :
               Enrichie RNE_ qui est une variable global
     
     """
    
    #Dictionnaire de travail, externe à la fonction
    global RNE_
    
    #Récupération des clefs itératives de RNE_
    autre_ape = [f"autre_{i}_ape" for i in range(1,11)]
    autre_statutFormalites = [f"autre_{i}_statutFormalites" for i in range(1,11)]
    autre_role = [f"autre_{i}_role" for i in range(1,11)]
    autre_adresse = [f"autre_{i}_adresse" for i in range(1,11)]
    autre_datefin = [f"autre_{i}_datefin" for i in range(1,11)]
    autre_dateeffetferm = [f"autre_{i}_dateeffetferm" for i in range(1,11)]
    autre_datedebut = [f"autre_{i}_datedebut" for i in range(1,11)]
    autre_dateeffettrans = [f"autre_{i}_dateeffettrans" for i in range(1,11)]
    
    associe_role = [f"associe_{i}_role" for i in range(1,6)]
    associe_nom = [f"associe_{i}_nom" for i in range(1,6)]
    associe_prenom = [f"associe_{i}_prenom" for i in range(1,6)]
    associe_naissance = [f"associe_{i}_naissance" for i in range(1,6)]
    associe_id = [f"associe_{i}_id" for i in range(1,6)]
    
    #Premiers get des json
    formality = firme.get("formality")
    content = formality.get("content")

                  #########################################################################################
            ######    Parcours des arborescences de chaque dictionnaire "content" pour une firme donnée    #######
                  #########################################################################################
    
    # --- Informations générales indépendantes de personnes physiques ou morale ---
    cessation = content.get("natureCessation")
    RNE_["cessation"].append(1 if cessation else 0)
    
    RNE_["siren"].append(formality.get("siren"))
    RNE_["date_creation"].append(content.get("natureCreation", {}).get("dateCreation"))
    RNE_["etranger"].append(content.get("natureCreation", {}).get("societeEtrangere"))
    RNE_["micro"].append(content.get("natureCreation", {}).get("microEntreprise"))
    RNE_["agricole"].append(content.get("natureCreation", {}).get("entrepriseAgricole"))
    RNE_["eirl"].append(content.get("natureCreation", {}).get("eirl"))
    RNE_["formeJuridique"].append(content.get("natureCreation", {}).get("formeJuridique","non-disp"))
    
    personneMorale = content.get("personneMorale")
    personnePhysique = content.get("personnePhysique")
    
    # ===================================================================
    # CAS 1 : PERSONNE MORALE
    # ===================================================================
    if personneMorale is not None:
        RNE_["personneMorale"].append(1)
    
        # --- Adresse ---
        adresse = personneMorale.get("adresseEntreprise", {}).get("adresse", {})
        RNE_["code_postal"].append(adresse.get("codePostal", "non-disp"))
        RNE_["voie"].append(adresse.get("voie", "non-disp"))
        RNE_["codeInseeCommune"].append(adresse.get("codeInseeCommune", "non-disp"))
        RNE_["numVoie"].append(adresse.get("numVoie", "non-disp"))
        RNE_["typeVoie"].append(adresse.get("typeVoie", "non-disp"))
    
        # --- Information sur la cessation si cessation ---
        cessation_info = personneMorale.get("detailCessationEntreprise", {})
        RNE_["dateEffet"].append(cessation_info.get("dateEffet", "non-disp"))
        RNE_["dateRadiation"].append(cessation_info.get("dateRadiation", "non-disp"))
        RNE_["indicateurDecesEntrepreneur"].append(cessation_info.get("indicateurDecesEntrepreneur", "non-disp"))
        RNE_["dateCessationTotaleActivite"].append(cessation_info.get("dateCessationTotaleActivite", "non-disp"))
        RNE_["dateClotureLiquidation"].append(cessation_info.get("dateClotureLiquidation", "non-disp"))
        RNE_["dateDissolutionDisparition"].append(cessation_info.get("dateDissolutionDisparition", "non-disp"))
        RNE_["indicateurCessationTemporaire"].append(cessation_info.get("indicateurCessationTemporaire", "non-disp"))
        RNE_["indicateurPoursuiteActivite"].append(cessation_info.get("indicateurPoursuiteActivite", "non-disp"))
        RNE_["indicateurMaintienImmatriculationRegistre"].append(cessation_info.get("indicateurMaintienImmatriculationRegistre", "non-disp"))
        RNE_["indicateurDissolution"].append(cessation_info.get("indicateurDissolution", "non-disp"))
        RNE_["typeDissolution"].append(cessation_info.get("typeDissolution", "non-disp"))
        RNE_["indicateurDisparitionPM"].append(cessation_info.get("indicateurDisparitionPM", "non-disp"))
        RNE_["motifDisparition"].append(cessation_info.get("motifDisparition", "non-disp"))
        RNE_["dateMiseEnSommeil"].append(cessation_info.get("dateMiseEnSommeil", "non-disp"))
        
        # --- Information sur l'établissement principal si existence + historique (statutPourFormalitePrincipal) ---
        principal = personneMorale.get("etablissementPrincipal")
        RNE_["rolePourEntreprisePrincipal"].append(principal.get("descriptionEtablissement").get("rolePourEntreprise","non-disp") if principal is not None else "principal_false")
        RNE_["statutPourFormalitePrincipal"].append(principal.get("descriptionEtablissement").get("statutPourFormalite","non-disp") if principal is not None else "principal_false")
        RNE_["principalAPEPrincipal"].append(principal.get("descriptionEtablissement").get("codeAPE","non-disp") if principal is not None else "principal_false")
        RNE_["principalDateTransPrincipal"].append(principal.get("descriptionEtablissement").get("dateEffetTransfert","non-disp") if principal is not None else "pincipal_false_transfert")
        
        # --- Information sur les établissements secondaires si existence + historique ---
        autres = personneMorale.get("autresEtablissements")
        if autres is not None:
            
            RNE_["nb_autres"].append(len(autres))
            
            i = -1
            for i in range(min(len(autres),10)):
              
                ape_autre = autre_ape[i]
                statutFormalites_autre = autre_statutFormalites[i]
                role_autre = autre_role[i]
                adresse_autre = autre_adresse[i]
                datefin_autre = autre_datefin[i]
                dateeffetferm_autre = autre_dateeffetferm[i]
                datedebut_autre = autre_datedebut[i]
                dateeffettrans_autre = autre_dateeffettrans[i]
                
                RNE_[ape_autre].append(autres[i].get("descriptionEtablissement").get("codeApe","non-disp")
                                              if autres[i].get("descriptionEtablissement") else "non-disp")
                
                RNE_[statutFormalites_autre].append(autres[i].get("descriptionEtablissement").get("statutPourFormalite","non-disp")
                                              if autres[i].get("descriptionEtablissement") else "non-disp")
                
                RNE_[role_autre].append(autres[i].get("descriptionEtablissement").get("rolePourEntreprise","non-disp")
                                              if autres[i].get("descriptionEtablissement") else "non-disp")
                
                RNE_[adresse_autre].append(autres[i].get("adresse").get("codePostal","non-disp")
                                              if autres[i].get("adresse") else "non-disp")
                
                RNE_[datedebut_autre].append(autres[i].get("activites")[0].get("dateDebut","non-disp")
                                              if len(autres[i].get("activites")) > 0  else "non-disp")
                
                RNE_[datefin_autre].append(autres[i].get("activites")[0].get("dateFin","actif") 
                                              if len(autres[i].get("activites")) > 0 
                                                    else "non-disp")
                
                RNE_[dateeffetferm_autre].append(autres[i].get("descriptionEtablissement").get("dateEffetFermeture","non-disp")
                                              if autres[i].get("descriptionEtablissement") else "non-disp")
                
                RNE_[dateeffettrans_autre].append(autres[i].get("descriptionEtablissement").get("dateEffetTransfert","non-disp")
                                              if autres[i].get("descriptionEtablissement") else "non-disp")
                
                
            if -1 <= i < 9:
                for j in range(i+1,10):
                    
                    ape_autre = autre_ape[j]
                    statutFormalites_autre = autre_statutFormalites[j]
                    role_autre = autre_role[j]
                    adresse_autre = autre_adresse[j]
                    datefin_autre = autre_datefin[j]
                    dateeffetferm_autre = autre_dateeffetferm[j]
                    datedebut_autre = autre_datedebut[j]
                    dateeffettrans_autre = autre_dateeffettrans[j]
                    
                    
                    RNE_[ape_autre].append("existence_null")
                    RNE_[statutFormalites_autre].append("existence_null")
                    RNE_[role_autre].append("existence_null")
                    RNE_[adresse_autre].append("existence_null")
                    RNE_[datedebut_autre].append("existence_null")
                    RNE_[datefin_autre].append("existence_null")
                    RNE_[dateeffetferm_autre].append("existence_null")
                    RNE_[dateeffettrans_autre].append("existence_null")
                
        else:
            
               for j in range(10):
                   
                   ape_autre = autre_ape[j]
                   statutFormalites_autre = autre_statutFormalites[j]
                   role_autre = autre_role[j]
                   adresse_autre = autre_adresse[j]
                   datefin_autre = autre_datefin[j]
                   dateeffetferm_autre = autre_dateeffetferm[j]
                   datedebut_autre = autre_datedebut[j]
                   dateeffettrans_autre = autre_dateeffettrans[j]
                   
                   
                   RNE_[ape_autre].append("existence_null")
                   RNE_[statutFormalites_autre].append("existence_null")
                   RNE_[role_autre].append("existence_null")
                   RNE_[adresse_autre].append("existence_null")
                   RNE_[datedebut_autre].append("existence_null")
                   RNE_[datefin_autre].append("existence_null")
                   RNE_[dateeffetferm_autre].append("existence_null")
                   RNE_[dateeffettrans_autre].append("existence_null")
            
        
        
    
        # --- Identité entreprise ---
        identite = personneMorale.get("identite", {})
        entreprise = identite.get("entreprise", {})
        description = identite.get("description", {})
    
        RNE_["denomination"].append(entreprise.get("denomination", "non-disp"))
        RNE_["nicSiege"].append(entreprise.get("nicSiege", "non-disp"))
        RNE_["codeAPE"].append(entreprise.get("codeApe", "non-disp"))
        RNE_["dateImmat"].append(entreprise.get("dateImmat", "non-disp"))
        RNE_["dateDebutAct"].append(entreprise.get("dateDebutActiv", "non-disp"))
    
        RNE_["objet"].append(description.get("objet", "non-disp"))
        RNE_["capitalVariable"].append(description.get("capitalVariable", "non-disp"))
        RNE_["montantCapital"].append(description.get("montantCapital", "non-disp"))
        
        # --- Informations sur les associés au pouvoir ---
        compo = personneMorale.get("composition")
        
        if compo is not None:
            pouvoirs = compo.get("pouvoirs")
            
          
            i = -1
            for i in range(min(len(pouvoirs),5)):
                  
                  role_associe = associe_role[i]
                  nom_associe = associe_nom[i]
                  prenom_associe = associe_prenom[i]
                  id_associe = associe_id[i]
                  naissance_associe = associe_naissance[i]
                  
                  if pouvoirs[i].get("typeDePersonne") == "INDIVIDU":
                      
                      RNE_[role_associe].append(pouvoirs[i].get("roleEntreprise","non-disp"))
                      
                      RNE_[nom_associe].append(pouvoirs[i].get("individu").get("descriptionPersonne").get("nom")
                                              if pouvoirs[i].get("individu") else "non-disp")
                      
                      prenoms_is = pouvoirs[i].get("individu").get("descriptionPersonne").get("prenoms")
    
                      if isinstance(prenoms_is,list) and len(prenoms_is) > 0 :
                          RNE_[prenom_associe].append(pouvoirs[i].get("individu").get("descriptionPersonne").get("prenoms")[0]
                                              if (pouvoirs[i].get("individu")) else "non-disp")
                      else:
                          RNE_[prenom_associe].append("non-disp")
                      
                      RNE_[naissance_associe].append(pouvoirs[i].get("individu").get("descriptionPersonne").get("dateDeNaissance")
                                              if pouvoirs[i].get("individu") else "non-disp")
                      
                      RNE_[id_associe].append(pouvoirs[i].get("representantId"))
                      
                  else:
                      
                      RNE_[role_associe].append("MORALE")
                      RNE_[nom_associe].append("MORALE")
                      RNE_[prenom_associe].append("MORALE")
                      RNE_[naissance_associe].append("MORALE")
                      RNE_[id_associe].append("MORALE")
            
            if -1 <= i < 4:
                for j in range(i+1,5):
                      
                      role_associe = associe_role[j]
                      nom_associe = associe_nom[j]
                      prenom_associe = associe_prenom[j]
                      naissance_associe = associe_naissance[j]
                      id_associe = associe_id[j]
                      
                      RNE_[role_associe].append("existence_null")
                      RNE_[nom_associe].append("existence_null")
                      RNE_[prenom_associe].append("existence_null")
                      RNE_[naissance_associe].append("existence_null")
                      RNE_[id_associe].append("existence_null")
                      
        else:
            for k in range(5):
                        
                  role_associe = associe_role[k]
                  nom_associe = associe_nom[k]
                  prenom_associe = associe_prenom[k]
                  naissance_associe = associe_naissance[k]
                  id_associe = associe_id[k]
                  
                  RNE_[role_associe].append("existence_null")
                  RNE_[nom_associe].append("existence_null")
                  RNE_[prenom_associe].append("existence_null")
                  RNE_[naissance_associe].append("existence_null")
                  RNE_[id_associe].append("existence_null")
            
                    
                    
        # --- Est dans personne physique mais pas morale --- 
        for v in ["nom", "prenoms", "genre","age_physique"]:
            RNE_[v].append("non-disp")
    
    # ===================================================================
    # CAS 2 : PERSONNE PHYSIQUE
    # ===================================================================
    elif personnePhysique is not None:
        RNE_["personneMorale"].append(0)
    
        # --- Adresse ---
        adresse = personnePhysique.get("adresseEntreprise", {}).get("adresse", {})
        RNE_["code_postal"].append(adresse.get("codePostal", "non-disp"))
        RNE_["voie"].append(adresse.get("voie", "non-disp"))
        RNE_["codeInseeCommune"].append(adresse.get("codeInseeCommune", "non-disp"))
        RNE_["numVoie"].append(adresse.get("numVoie", "non-disp"))
        RNE_["typeVoie"].append(adresse.get("typeVoie", "non-disp"))
    
        # --- APE nicsiege dateimmat datedévutact formejuridique ---
        firme_info = personnePhysique.get("identite", {}).get("entreprise", {})
        RNE_["codeAPE"].append(firme_info.get("codeApe","non-disp"))
        RNE_["nicSiege"].append(firme_info.get("nicSiege","non-disp"))
        RNE_["dateImmat"].append(firme_info.get("dateImmat", "non-disp"))
        RNE_["dateDebutAct"].append(firme_info.get("dateDebutActiv", "non-disp"))
        
        
        # --- Information sur la cessation si cessation ---
        cessation_info = personnePhysique.get("detailCessationEntreprise", {})
        RNE_["dateEffet"].append(cessation_info.get("dateEffet", "non-disp"))
        RNE_["dateRadiation"].append(cessation_info.get("dateRadiation", "non-disp"))
        RNE_["indicateurDecesEntrepreneur"].append(cessation_info.get("indicateurDecesEntrepreneur", "non-disp"))
        RNE_["dateCessationTotaleActivite"].append(cessation_info.get("dateCessationTotaleActivite", "non-disp"))
        RNE_["dateClotureLiquidation"].append(cessation_info.get("dateClotureLiquidation", "non-disp"))
        RNE_["dateDissolutionDisparition"].append(cessation_info.get("dateDissolutionDisparition", "non-disp"))
        RNE_["indicateurCessationTemporaire"].append(cessation_info.get("indicateurCessationTemporaire", "non-disp"))
        RNE_["indicateurPoursuiteActivite"].append(cessation_info.get("indicateurPoursuiteActivite", "non-disp"))
        RNE_["indicateurMaintienImmatriculationRegistre"].append(cessation_info.get("indicateurMaintienImmatriculationRegistre", "non-disp"))
        RNE_["indicateurDissolution"].append(cessation_info.get("indicateurDissolution", "non-disp"))
        RNE_["typeDissolution"].append(cessation_info.get("typeDissolution", "non-disp"))
        RNE_["indicateurDisparitionPM"].append(cessation_info.get("indicateurDisparitionPM", "non-disp"))
        RNE_["motifDisparition"].append(cessation_info.get("motifDisparition", "non-disp"))
        RNE_["dateMiseEnSommeil"].append(cessation_info.get("dateMiseEnSommeil", "non-disp"))
        
        
        # --- Information sur l'établissement principal si existence + historique (statutPourFormalitePrincipal) ---
        principal = personnePhysique.get("etablissementPrincipal")
        RNE_["rolePourEntreprisePrincipal"].append(principal.get("descriptionEtablissement").get("rolePourEntreprise","non-disp") 
        if principal is not None else "principal_false")
        
        RNE_["statutPourFormalitePrincipal"].append(principal.get("descriptionEtablissement").get("statutPourFormalite","non-disp") 
        if principal is not None else "principal_false")
    
        RNE_["principalAPEPrincipal"].append(principal.get("descriptionEtablissement").get("codeAPE","non-disp") 
        if principal is not None else "principal_false")
    
        RNE_["principalDateTransPrincipal"].append(principal.get("descriptionEtablissement").get("dateEffetTransfert","non-disp") 
        if principal is not None else "pincipal_false_transfert")
        
        # --- Information sur les établissements secondaires si existence + historique ---
        autres = personnePhysique.get("autresEtablissements")
        if autres is not None:
            
            RNE_["nb_autres"].append(len(autres))
            
            i = -1
            
            for i in range(min(len(autres),10)):
              
                ape_autre = autre_ape[i]
                statutFormalites_autre = autre_statutFormalites[i]
                role_autre = autre_role[i]
                adresse_autre = autre_adresse[i]
                datefin_autre = autre_datefin[i]
                dateeffetferm_autre = autre_dateeffetferm[i]
                datedebut_autre = autre_datedebut[i]
                dateeffettrans_autre = autre_dateeffettrans[i]
                
                RNE_[ape_autre].append(autres[i].get("descriptionEtablissement").get("codeApe","non-disp"))
                RNE_[statutFormalites_autre].append(autres[i].get("descriptionEtablissement").get("statutPourFormalite","non-disp"))
                RNE_[role_autre].append(autres[i].get("descriptionEtablissement").get("rolePourEntreprise","non-disp"))
                RNE_[adresse_autre].append(autres[i].get("adresse").get("codePostal","non-disp"))
                RNE_[datedebut_autre].append(autres[i].get("activites")[0].get("dateDebut","non-disp") if len(autres[i].get("activites")) > 0 
                                                    else "non-disp")
                RNE_[datefin_autre].append(autres[i].get("activites")[0].get("dateFin","actif") if len(autres[i].get("activites")) > 0 
                                                    else "non-disp")
                RNE_[dateeffetferm_autre].append(autres[i].get("descriptionEtablissement").get("dateEffetFermeture","non-disp"))
                RNE_[dateeffettrans_autre].append(autres[i].get("descriptionEtablissement").get("dateEffetTransfert","non-disp"))
            
            if -1 <= i < 9:
                
                for j in range(i+1,10):
                    
                    ape_autre = autre_ape[j]
                    statutFormalites_autre = autre_statutFormalites[j]
                    role_autre = autre_role[j]
                    adresse_autre = autre_adresse[j]
                    datefin_autre = autre_datefin[j]
                    dateeffetferm_autre = autre_dateeffetferm[j]
                    datedebut_autre = autre_datedebut[j]
                    dateeffettrans_autre = autre_dateeffettrans[j]
                    
                    
                    RNE_[ape_autre].append("existence_null")
                    RNE_[statutFormalites_autre].append("existence_null")
                    RNE_[role_autre].append("existence_null")
                    RNE_[adresse_autre].append("existence_null")
                    RNE_[datedebut_autre].append("existence_null")
                    RNE_[datefin_autre].append("existence_null")
                    RNE_[dateeffetferm_autre].append("existence_null")
                    RNE_[dateeffettrans_autre].append("existence_null")
                    
        else:
            for j in range(10):
                
                ape_autre = autre_ape[j]
                statutFormalites_autre = autre_statutFormalites[j]
                role_autre = autre_role[j]
                adresse_autre = autre_adresse[j]
                datefin_autre = autre_datefin[j]
                dateeffetferm_autre = autre_dateeffetferm[j]
                datedebut_autre = autre_datedebut[j]
                dateeffettrans_autre = autre_dateeffettrans[j]
                
                
                RNE_[ape_autre].append("existence_null")
                RNE_[statutFormalites_autre].append("existence_null")
                RNE_[role_autre].append("existence_null")
                RNE_[adresse_autre].append("existence_null")
                RNE_[datedebut_autre].append("existence_null")
                RNE_[datefin_autre].append("existence_null")
                RNE_[dateeffetferm_autre].append("existence_null")
                RNE_[dateeffettrans_autre].append("existence_null")
        
        
        
        # --- Identité de la personne physique ---
        identite = personnePhysique.get("identite", {}).get("entrepreneur", {}).get("descriptionPersonne", {})
        RNE_["nom"].append(identite.get("nom", "non-disp"))
        RNE_["prenoms"].append(identite.get("prenoms")[0] if isinstance(identite.get("prenoms"),list) and len(identite.get("prenoms"))>0 else 'non-disp')
        RNE_["genre"].append(identite.get("genre", "non-disp"))
        RNE_["age_physique"].append(identite.get("dateDeNaissance","non-disp"))
        
        # --- objet ---
        
        
        RNE_["objet"].append(personnePhysique.get("etablissementPrincipal").get("activites")[0].get("descriptionDetaillee") if 
        (personnePhysique.get("etablissementPrincipal") is not None) and (isinstance(personnePhysique.get("etablissementPrincipal").get("activites"),list)) and (len(personnePhysique.get("etablissementPrincipal").get("activites")) >0) else "non-disp")
        
        # --- Est dans personne morale mais pas physique --- 
        RNE_["capitalVariable"].append("non-disp")
        RNE_["montantCapital"].append("non-disp")
        RNE_["denomination"].append("non-disp")
        
        for j in range(0,5):
              
              role_associe = associe_role[j]
              nom_associe = associe_nom[j]
              prenom_associe = associe_prenom[j]
              naissance_associe = associe_naissance[j]
              id_associe = associe_id[j]
              
              RNE_[role_associe].append("physique")
              RNE_[nom_associe].append("physique")
              RNE_[prenom_associe].append("physique")
              RNE_[naissance_associe].append("physique")
              RNE_[id_associe].append("physique")
    
    
    # --- Forme d’exercice , diffusion Insee et commerciale ---
    RNE_["formeExerciceActivitePrincipale"].append(content.get("formeExerciceActivitePrincipale", "non-disp"))
    RNE_["diffusionINSEE"].append(content.get("diffusionINSEE", "non-disp"))
    RNE_["diffusionCommerciale"].append(content.get("diffusionCommerciale", "non-disp"))

          


Nous partitionnons la récupération en plusieurs intervalles de temps, afin de ne pas saturer la mémoire 
lors de la mise en forme des data frame avec pandas, et d'autre part pour ne pas se retrouver avec des fichiers parquet trop gros, bien que ce format soit plus adapté et optimisé pour des grosses volumétries de données. L'exécution de cette cellule peut prendre du temps...

In [ ]:
MY_BUCKET = "guillaume176"

In [ ]:
dates_select = [1980,1990,2000,2005,2010,2015,2017,2018,2019,2020,2021,2022,2023,2024]

for i in range(1,len(dates_select)):

    df_name = f"RNE_{dates_select[i-1]}_{dates_select[i]}.parquet"

    FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/data_RNE/{df_name}"

    print(f"itération {i}/{len(dates_select)-1}, préparation de {df_name} \n")

    date_min = dates_select[i-1]
    date_max = dates_select[i]
    
    RNE_ = prepare_RNE_Dict()
    for firme_it in get_RNE_past([date_min,1,1], [date_max,1,1]):
        get_RNE_DataFrame(firme_it)

    data = pd.DataFrame(RNE_)
    data = data.astype(str)

    with fs.open(FILE_PATH_OUT_S3,"wb") as file_out:
        data.to_parquet(file_out, index=False)

    print(f"Data frame {df_name} prêt sur S3 \n")
    print(f"{fs.ls("guillaume176/diffusion/data_RNE")[-1]} \n")
    

100%|██████████| 13098/13098 [00:00<00:00, 3477024.92it/s]


itération 1/14, préparation de RNE_1990_2000.parquet 

Data frame RNE_1990_2000.parquet prêt sur S3 

guillaume176/diffusion/data_RNE/RNE_1980_1990.parquet 

itération 2/14, préparation de RNE_1990_2000.parquet 



100%|██████████| 13098/13098 [49:30<00:00,  4.41it/s] 


Data frame RNE_1990_2000.parquet prêt sur S3 

guillaume176/diffusion/data_RNE/RNE_1990_2000.parquet 

itération 3/14, préparation de RNE_2000_2005.parquet 



100%|██████████| 13098/13098 [43:51<00:00,  4.98it/s] 


...

Data frame RNE_2020_2021.parquet prêt sur S3 

guillaume176/diffusion/data_RNE/RNE_2020_2021.parquet 

itération 11/14, préparation de RNE_2021_2022.parquet 



100%|██████████| 13098/13098 [37:15<00:00,  5.86it/s] 


Data frame RNE_2021_2022.parquet prêt sur S3 

guillaume176/diffusion/data_RNE/RNE_2021_2022.parquet 

itération 12/14, préparation de RNE_2022_2023.parquet 



100%|██████████| 13098/13098 [37:37<00:00,  5.80it/s] 


Data frame RNE_2022_2023.parquet prêt sur S3 

guillaume176/diffusion/data_RNE/RNE_2022_2023.parquet 



### Voici une liste synthétique des variables obtenues : 


## Description
153 Variables :
## 1. Informations générales sur l’entreprise
- `siren` : numéro SIREN  
- `denomination` : nom de l’entreprise  
- `formeJuridique` : forme juridique (SARL, SAS…)  
- `date_creation` : date de création  
- `dateImmat` : date d’immatriculation  
- `dateCessationTotaleActivite` : date de cessation totale  
- `dateRadiation` : date de radiation  
- `dateClotureLiquidation` : date de clôture liquidation  
- `dateDissolutionDisparition` : date dissolution/disparition  
- `motifDisparition` : motif de disparition  
- `capitalVariable` : indicateur capital variable  
- `montantCapital` : montant du capital  
- `objet` : objet social  
- `codeAPE` : code APE principal  
- `formeExerciceActivitePrincipale` : forme d’exercice de l’activité principale  
- `personneMorale` : indicateur si entreprise est personne morale  
- `nicSiege` : NIC du siège  
- `etranger` : indicateur entreprise étrangère  
- `micro` : indicateur micro-entreprise  
- `agricole` : indicateur activité agricole  
- `eirl` : indicateur EIRL  
- `code_postal`, `codeInseeCommune` : localisation  
- `voie`, `typeVoie`, `numVoie` : adresse du siège  
- `cessation` : indicateur de cessation d’activité  

---

## 2. Activité et formalité principale
- `dateDebutAct` : date début activité  
- `rolePourEntreprisePrincipal` : rôle dans l’entreprise  
- `statutPourFormalitePrincipal` : statut pour formalité principale  
- `principalAPEPrincipal` : code APE principal  
- `principalDateTransPrincipal` : date de transmission principale  
- `indicateurDecesEntrepreneur` : indicateur décès entrepreneur  
- `indicateurCessationTemporaire` : indicateur de cessation temporaire  
- `indicateurPoursuiteActivite` : indicateur poursuite d’activité  
- `indicateurMaintienImmatriculationRegistre` : indicateur maintien immatriculation  
- `indicateurDissolution` : indicateur dissolution  
- `indicateurDisparitionPM` : indicateur disparition personne morale  
- `dateMiseEnSommeil` : date mise en sommeil  
- `typeDissolution` : type de dissolution  
- `nb_autres` : nombre d’autres établissements
- `motifDisparition` : motif de la disparition de l'entreprise
- `diffusionCommerciale` : autorisation diffusion commerciale
- `diffusionINSEE` : autorisation diffusion Insee
---

## 3. Autres activités / formalités secondaires
Variables répétitives pour `i` allant de 1 à 10 :  
- `autre_i_ape` : code APE secondaire  
- `autre_i_statutFormalites` : statut de la formalité secondaire  
- `autre_i_role` : rôle dans l’activité secondaire  
- `autre_i_adresse` : adresse de l’activité secondaire  
- `autre_i_datefin` : date de fin de l’activité  
- `autre_i_dateeffetferm` : date effet fermeture secondaire
- `autre_i_datedebut` : date début de l’activité secondaire
- `autre_i_dateeffettrans` : date effet transmission

---

## 4. Associés (personnes morale)
Variables répétitives pour `i` allant de 1 à 5 :  
- `associe_i_id` : identifiant interne de l’associé  
- `associe_i_role` : rôle dans l’entreprise de l'associé  
- `associe_i_nom` : nom associé
- `associe_i_prenom` : prénom associé 
- `associe_i_naissance` : date de naissance associé

## 5. Informations personnes physiques
- `nom` : nom de la personne physique
- `prenom`: prénom de la personne physique
- `age_physique` : age de la personne physique


## Fichiers fournis sur le ssp cloud S3 guillaume176/diffusion/data_RNE: 
- `RNE_1980_1990.parquet`
- `RNE_1990_2000.parquet`
- `RNE_2000_2005.parquet`
- `RNE_2005_2010.parquet`
- `RNE_2010_2015.parquet`
- `RNE_2015_2017.parquet`
- `RNE_2017_2018.parquet`
- `RNE_2018_2019.parquet`
- `RNE_2019_2020.parquet`
- `RNE_2020_2021.parquet`
- `RNE_2021_2022.parquet`
- `RNE_2022_2023.parquet`
- `RNE_2023_2024.parquet`

In [39]:
path_data = "guillaume176/diffusion/data_RNE"
file_list = fs.ls(path_data)[1:]

file_list = ["https://minio.lab.sspcloud.fr/" + f for f in file_list]

In [ ]:
#Liens de téléchargement des données
file_list

['https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1980_1990.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1990_2000.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2000_2005.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2005_2010.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2010_2015.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2015_2017.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2017_2018.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2018_2019.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2019_2020.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2020_2021.parquet',
 'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2021_2022.parquet',
 'https://minio.lab.sspcloud.fr/

-------------------------------------------

## I.1.2 - Récupération des entreprises crées en IDF depuis 2010 jusqu'en 2024 pour les secteurs du commerce, de la restauration et de l'hôtellerie ; Récupération des expériences passés des entrepreneurs : 

Le but de cette partie est d'obtenir un data frame sur lequel nous travaillerons tout au long de ce projet. Nous décidons de récupérer l'ensemble des entreprises françaises crées entre 2010 et 2024, dont l'adresse principale est en Ile de France, pour les secteurs du commerce, de l'hôtellerie et la restauration. 

Notre choix s'explique par le fait que récupérer des entreprises crées à des dates trop lointaines pourraient fausser les résultats de la modélisation choisie, puisque par exemple, le tissu économique français des années 80 est drastiquement différent de celui d'aujourd'hui. 
De même, récupérer trop de secteurs différents pourrait entraîner une incapacité du modèle que l'on choisira à discriminer entre les entreprises radiées et non-radiées, qui sera notre variable cible à prédire.

Le choix de l'Ile de France s'explique aussi pour les mêmes raisons.


--------------------
Nous donnons ici une suite de fonction permettant de récupérer ce data frame. Nous récupérons au passage des variables concernant l'expérience potentielle des associés et des personnes physiques, qui nécessitent un traitement assez lourd. Pour simplifier les choses, on considéra qu'une personne physique est l'associé 0.

Voici la stratégie retenue pour n'obtenir qu'un seul data frame exploitable.

- La fonction `get_RNE_Exp` permet de récupérer l'expérience des associés. Cette fonction est la raison principale pour laquelle nous incluons cette partie de notre travail dans la partie récupération des données, et non dans celle de l'analyse et feature engineering.
En effet, nous voulons pour chaque entreprise d'IDF crées de 2010 à 2024 dans les secteurs visés, compter le nombre de fois où chaque associé à été enrôlé ou non dans une entreprise passé, c'est-à-dire crées avant la date de création concernant l'entreprise que nous regardons. Nous voulons aussi n'effectuer ce comptage que si l'associé à eu un rôle pertinent dans l'entreprise, le rendant le plus susceptible d'être à l'origine de la création bien que cela ne soit pas garanti. Enfin, nous voulons également compter le nombre d'entreprise radiée parmi les entreprises dans lesquelles l'associé en question à été enrôlé. Nous pensons en effet que ces variables permettront d'enrichir en information notre base de données, donnant plus de chance à un modèle de machine learning d'aboutir à des métriques satisfaisantes.
Pour effectuer cela, nous voulons travailler avec deux data frames : l'un dit de "travaille", étant celui sur lequel nous nous concentrons (idf_10_24, appelé simplement df_p dans la fonction), l'autre dit "historique" (total_p), comprenant l'ensemble des entreprises crées partout en France, pour les "grands" secteurs du commerce, artisanat et artisanat réglementé, depuis 1980 jusqu'en 2024. Pour chaque entreprise dans idf_10_24, nous regardons si le combo nom + prénom + date de naissance pour un associé donné correspond à une entreprise crées dans le passé. Si oui, nous sélectionnons seulement les entreprises dans lesquels l'associé à eu un rôle important. Nous récupérons au passage le nombre d'entreprises passées radiées. 
Comme cette tâche nécessite de travailler avec un data frame de plus de 8 000 000 de lignes (total), pandas n'est pas adapté ici. Nous utilisons alors DuckDB permettant de faire des requêtes SQL simple et rapide. 
`On récupère ainsi les variables total_expi (expérience total de l'associé i), total_radi (nombre d'entreprises passés radiées liées à l'associé i) et ape_truei (nombre d'entreprises passés ayant un lien avec le secteur d'activité de l'entreprise actuelle liée à l'associé i).`


Remarque : l'id (nom + prénom + date de naissance) est ici utilisé par défaut puisque les variables "associe_i_id" ne trouvent pas de correspondance dans le passé. Elles sont utiles simplement à l'identification interne d'un associé dans une entreprise donné.
D'autre part, il se peut que deux personnes différentes partagent cet id, même si cela est rare. Ceci sera un point d'amélioration à considérer.

- La fonction `get_RNE_MeanAge` permet de récupérer l'âge moyen des associées ainsi que le nombre d'associé.


- La fonction `get_RNE_Final` permet de récupérer les données depuis le cloud ssp cloud S3 puis les préparer pour la fonction `get_RNE_Exp` ainsi pouvoir récupérer l'expérience de chaque associé. Elle récupère ensuite l'âge moyen des associés et le nombre d'associé grâce à la fonction `get_RNE_MeanAge`. Elle renvoie un data frame pandas. Cette fonction se veux flexible pour que les lecteurs de ce projet puissent récupérer d'autres années, départements, activités...

- Finalement, nous exportons au format parquet le data frame obtenu sur le cloud, qui servira à la deuxième partie de la récupération de données.
Voire plus loin pour le lien de téléchargement de ce data frame.

In [2]:

def get_RNE_Exp(numero_associe : int, df_p : pd.DataFrame, total_p : pd.DataFrame) -> pd.DataFrame:

    """
    Fonction qui permet de récupérer l'expérience d'un associée donné dont le numéro est compris en 1 et 5 ou bien celle
    d'une personne physique (0).

    
    inputs : 
            - numero_associe : int compris en entre 0 et 5 (0 == personne physique)
            - df_p : le dataframe RNE pour lequel on veut obtenir l'expérience des associés/personnes physiques
            - total_p : le dataframe RNE complet servant d'historique

    outputs :
            - res : dataframe avec 4 colonnes : siren, total_expi, total_radi, ape_truei, dont siren corresponds aux siren de 
                    df.
    
    total_exp0, total_rad0 et ape_true0 concernent les personnes physiques



    """
    if numero_associe > 0:
        ass1_nom = f"associe_{numero_associe}_nom"
        ass1_prenom = f"associe_{numero_associe}_prenom"
        ass1_naissance = f"associe_{numero_associe}_naissance"
    else:
        ass1_nom = "nom"
        ass1_prenom = "prenoms"
        ass1_naissance = "age_physique"

    df = df_p[["siren","date_creation",ass1_nom,ass1_prenom,ass1_naissance,"associe_1_role","codeAPE"]].copy()

    total = (total_p[["siren","date_creation",
    "associe_1_nom","associe_1_prenom","associe_1_naissance","associe_1_role",
    "associe_2_nom","associe_2_prenom","associe_2_naissance","associe_2_role",
    "associe_3_nom","associe_3_prenom","associe_3_naissance","associe_3_role",
    "associe_4_nom","associe_4_prenom","associe_4_naissance","associe_4_role",
    "associe_5_nom","associe_5_prenom","associe_5_naissance","associe_5_role",
    "nom","prenoms","age_physique","dateRadiation","codeAPE"]]).copy()

    df[ass1_nom] = df[ass1_nom].str.lower()
    df[ass1_prenom] = df[ass1_prenom].str.lower()
    df[ass1_naissance] = df[ass1_naissance].str.lower()


    associe_nom = [f"associe_{i}_nom" for i in range(1,6)]
    associe_prenom = [f"associe_{i}_prenom" for i in range(1,6)]
    associe_naissance = [f"associe_{i}_naissance" for i in range(1,6)]
    for i in range(len(associe_nom)):
        total[associe_nom[i]] = total[associe_nom[i]].str.lower()
        total[associe_prenom[i]] = total[associe_prenom[i]].str.lower()
        total[associe_naissance[i]] = total[associe_naissance[i]].str.lower()

    total["nom"] = total["nom"].str.lower()
    total["prenoms"] = total["prenoms"].str.lower()
    total["age_physique"] = total["age_physique"].str.lower()

    df["siren"] = df["siren"].astype(int)
    total["siren"] = total["siren"].astype(int)

    total['dateRadiation'] = pd.to_datetime(total['dateRadiation'], errors='coerce')
    

    #Connexion DuckDB
    con = duckdb.connect()

    #Enregistrement des DataFrames pandas dans DuckDB
    con.register("df", df)        
    con.register("total", total)

    con.execute("""
    CREATE OR REPLACE VIEW total_long AS
    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_1_nom AS sname,
        associe_1_prenom AS fname,
        associe_1_naissance AS birth,
        associe_1_role AS role
    FROM total
    WHERE associe_1_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_1_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_2_nom,
        associe_2_prenom,
        associe_2_naissance,
        associe_2_role
    FROM total
    WHERE associe_2_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_2_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_3_nom,
        associe_3_prenom,
        associe_3_naissance,
        associe_3_role
    FROM total
    WHERE associe_3_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_3_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_4_nom,
        associe_4_prenom,
        associe_4_naissance,
        associe_4_role
    FROM total
    WHERE associe_4_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_4_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        associe_5_nom,
        associe_5_prenom,
        associe_5_naissance,
        associe_5_role
    FROM total
    WHERE associe_5_nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_prenom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_naissance NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND associe_5_role IN ('28','29','41','53','73','74','75','101','65','66')

    UNION ALL

    SELECT siren, date_creation,dateRadiation,codeAPE,
        nom,
        prenoms,
        age_physique,
        associe_5_role
    FROM total
    WHERE nom NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND prenoms NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique')
    AND age_physique NOT IN ('existence_null', 'non-disp', 'morale', 'none', 'physique');
    """)

    if numero_associe > 0:
        #Requête principale pour l'historique d'un associé
        res = con.execute(f"""
        SELECT
            df.siren,


            COUNT(DISTINCT t.siren) AS total_exp{numero_associe},


            COUNT(DISTINCT CASE
                WHEN t.dateRadiation IS NOT NULL THEN t.siren
            END) AS total_rad{numero_associe},

            COUNT(DISTINCT CASE
                WHEN SUBSTR(t.codeAPE, 1, 1) = SUBSTR(df.codeAPE, 1, 1) THEN t.siren
            END) AS ape_true{numero_associe}

        FROM df

        LEFT JOIN total_long t
        ON df.associe_{numero_associe}_nom = t.sname
        AND df.associe_{numero_associe}_prenom = t.fname
        AND df.associe_{numero_associe}_naissance = t.birth
        AND t.date_creation < df.date_creation

        GROUP BY df.siren
        ORDER BY df.siren
        """).df()


    else:
        #Requête principale pour  l'historique d'une personne physique
        res = con.execute(f"""
        SELECT
            df.siren,


            COUNT(DISTINCT t.siren) AS total_exp{numero_associe},


            COUNT(DISTINCT CASE
                WHEN t.dateRadiation IS NOT NULL THEN t.siren
            END) AS total_rad{numero_associe},

            COUNT(DISTINCT CASE
                WHEN SUBSTR(t.codeAPE, 1, 1) = SUBSTR(df.codeAPE, 1, 1) THEN t.siren
            END) AS ape_true{numero_associe}

        FROM df

        LEFT JOIN total_long t
        ON df.nom = t.sname
        AND df.prenoms = t.fname
        AND df.age_physique = t.birth
        AND t.date_creation < df.date_creation

        GROUP BY df.siren
        ORDER BY df.siren
        """).df()

    return res



In [15]:
def get_RNE_MeanAge(df : pd.DataFrame) -> pd.DataFrame:

    """
    Fonction permettant de récupérer l'âge moyen des associés ainsi que le nombre d'associé pour chaque entreprise données.

    inputs : 
            - df : data frame RNE
    
    ouputs : 
            - df : data frame enrichi de l'âge moyen des associés ainsi que le nombre d'associé.
    """
    
    #Récupération du nombre d'associés
    df["is1"] = (~df["associe_1_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is2"] = (~df["associe_2_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is3"] = (~df["associe_3_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is4"] = (~df["associe_4_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["is5"] = (~df["associe_5_nom"].isin(["non-disp","MORALE","None","non-disp","physique","existence_null","morale","none"])).astype(int)
    df["personneMorale"] = df["personneMorale"].astype(int)

    df["nb_associe"] = df["is1"] + df["is2"]  + df["is3"] + df["is4"] + df["is5"] + (1-df["personneMorale"])
    
    #Récupération de l'âge moyen des associés à la création d'entreprises
    df["associe_1_naissance"] = (df["associe_1_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_1_naissance"] = pd.to_datetime(df["associe_1_naissance"])

    df["associe_2_naissance"] = (df["associe_2_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_2_naissance"] = pd.to_datetime(df["associe_2_naissance"])

    """
    #Corrige un bug natif
    i_error = df.loc[df["associe_3_naissance"] == "0997-08"]["associe_3_naissance"].index
    df[i_error, "associe_3_naissance"] = "1997-08"
    """

    df["associe_3_naissance"] = (df["associe_3_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_3_naissance"] = pd.to_datetime(df["associe_3_naissance"],errors='coerce')


    df["associe_4_naissance"] = (df["associe_4_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_4_naissance"] = pd.to_datetime(df["associe_4_naissance"])

    df["associe_5_naissance"] = (df["associe_5_naissance"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["associe_5_naissance"] = pd.to_datetime(df["associe_5_naissance"])

    df["age_physique"] = (df["age_physique"]
    .apply(lambda x : x if x not in ["MORALE","physique","existence_null","None","non-disp"] else pd.NaT))
    df["age_physique"] = pd.to_datetime(df["age_physique"])
 
    
    df["age0"] = (df["date_creation"] - df["age_physique"])
    df["age1"] = (df["date_creation"] - df["associe_1_naissance"])
    df["age2"] = (df["date_creation"] - df["associe_2_naissance"])
    df["age3"] = (df["date_creation"] - df["associe_3_naissance"])
    df["age4"] = (df["date_creation"] - df["associe_4_naissance"])
    df["age5"] = (df["date_creation"] - df["associe_5_naissance"])

   
    df["age0"] = df["age0"].dt.total_seconds() / (86400 * 365.25)
    df["age1"] = df["age1"].dt.total_seconds() / (86400 * 365.25)
    df["age2"] = df["age2"].dt.total_seconds() / (86400 * 365.25)
    df["age3"] = df["age3"].dt.total_seconds() / (86400 * 365.25)
    df["age4"] = df["age4"].dt.total_seconds() / (86400 * 365.25)
    df["age5"] = df["age5"].dt.total_seconds() / (86400 * 365.25)


    df["mean_age"] = (
        df[["age0","age1", "age2", "age3", "age4", "age5"]]
        .sum(axis=1, skipna=True)
        / (df["nb_associe"])
    )

    return df

In [16]:
def get_RNE_Final(dep : list, activites : list, date_min : list)-> pd.DataFrame:

    """
    Fonction permettant de récupérer les données du RNE sur le cloud ssp cloud S3, de récupérer les expériences passés des 
    associés, puis l'âge moyen des associés, ainsi que leur nombre, cela pour une liste de département et d'activités données,
    ainsi qu'une date minimale de création.

    inputs :
            - dep : liste de département de type ["78","01",...]
            - activites : liste d'activités par "bribes" de code APE de type ["45","5610B",...]
            - date_min : date minimale de création de type [2021,1,1] == [y,month,day]
    
    outputs : 
            - df_return : data frame RNE
    """


    ##################################################################################################
    #    ETAPE 1 : Récupération des données selon dep, acitivtes et date_min + l'historique total    #
    ##################################################################################################


    #Récupération des données depuis le cloud ssp cloud S3
    dates_select = [1980,1990,2000,2005,2010,2015,2017,2018,2019,2020,2021,2022,2023,2024]

    dfs_rne = dict()
    df_names = [(f"df_{dates_select[i-1]}_{dates_select[i]}",i) for i in range(1,len(dates_select))]

    url_dataRNE = ['https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1980_1990.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1990_2000.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2000_2005.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2005_2010.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2010_2015.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2015_2017.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2017_2018.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2018_2019.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2019_2020.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2020_2021.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2021_2022.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2022_2023.parquet',
    'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2023_2024.parquet']

    for names,i in tqdm(df_names):
        print(f"Récupération depuis le cloud {i}/13")
        url = url_dataRNE[i-1]
        df = pd.read_parquet(url)
        dfs_rne[names] = df
    #------------------------------------------------------------------------------------------------#

    #Récupération des colonnes et création d'un nouveau data frame vide
    columns_rne = list(dfs_rne[df_names[0][0]].columns)
    idf_10_24 = pd.DataFrame(columns=columns_rne)

    #Filtrage par activites et département
    for names in tqdm(df_names):
        print("Filtrage par département et activités")
        data = dfs_rne[names[0]]
        data = data[data["codeAPE"].str.startswith(tuple(activites)) & data["code_postal"].str.startswith(tuple(dep))]
        idf_10_24 = pd.concat([idf_10_24,data])
        
    #Filtrage pour les dates de création après 2010-01-01 (inlcus)
    idf_10_24["date_creation"] = pd.to_datetime(idf_10_24["date_creation"])

    print(f"Récupération depuis {date_min[0]}-{date_min[1]}-{date_min[2]}")
    idf_10_24 = idf_10_24[idf_10_24["date_creation"] >= datetime.datetime(date_min[0],date_min[1],date_min[2])]

    idf_10_24["dateRadiation"] = idf_10_24["dateRadiation"].apply(lambda x : 
        x if x not in ["non-disp","None"] else pd.NaT)


    #------------------------------------------------------------------------------------------------#

    #Récupération des colonnes et création d'un nouveau data frame vide
    columns_rne = list(dfs_rne[df_names[0][0]].columns)
    total = pd.DataFrame(columns=columns_rne)

    #Récupération complète dans un data frame
    print("Récupération de l'historique total")
    for names in tqdm(df_names):
        data = dfs_rne[names[0]]
        total = pd.concat([total,data])
    

    total["date_creation"] = pd.to_datetime(total["date_creation"])
    total["dateRadiation"] = total["dateRadiation"].apply(lambda x : 
        x if x not in ["non-disp","None"] else pd.NaT)

    #########################################################################################################
    #   ETAPE 2 : Récupération de l'expérience des associés/personnes physiques + age moyen à la création   #
    #########################################################################################################


    
    #Récupération de l'expérience des associés/personnes physiques
    associe_list = [0,1,2,3,4,5]

    idf_10_24["siren"] = idf_10_24["siren"].astype(int)

    df_return = idf_10_24.copy()

    for i in associe_list:
        
        print(f"Récupération expérience associé {i}")
        res = get_RNE_Exp(numero_associe=i, df_p=idf_10_24, total_p=total)
        res["siren"] = res["siren"].astype(int)
        df_return = df_return.merge(res, on="siren", how="left")
        
    #Récupération de l'âge moyen à la création

    print("Récupération de l'âge moyen")
    df_return = get_RNE_MeanAge(df_return)
    

    return df_return
    

Les cellules suivantes permettent de récupérer les données et de les exporter vers le cloud ssp cloud S3 :

In [17]:
#Listes des activités retenues par bribes d'APE (voire nomenclature NAF)
activites_annotees = [
    ("4511Z", "Commerce de voitures et véhicules légers"),
    ("4519Z", "Commerce d'autres véhicules automobiles"),
    ("45",    "Entretien et réparation véhicules automobiles"),
    ("454",   "Commerce et réparation de motocyclettes"),
    ("461",   "Intermédiaires du commerce de gros"),
    ("462",   "Commerce de gros produits agricoles et animaux vivants"),
    ("463",   "Commerce de gros aliments, boissons et tabac"),
    ("466",   "Commerce de gros autres équipements industriels"),
    ("467",   "Autres commerces de gros spécialisés"),
    ("471",   "Commerce de détail en magasin non spécialisé"),
    ("472",   "Commerce de détail alimentaire en magasin spécialisé"),
    ("473",   "Commerce de détail de carburants en magasin spécialisé"),
    ("478",   "Commerce de détail sur éventaires et marchés"),
    ("551",   "Hôtels et hébergement similaire"),
    ("552",   "Hébergement touristique et autre hébergement courte durée"),
    ("5610A", "Restauration traditionnelle"),
    ("5610B", "Cafétérias et autres libres-services"),
    ("5610C", "Restauration rapide"),
    ("563",    "Débits de boissons")
]

activites = [activites_annotees[i][0] for i in range(len(activites_annotees))]

#Départements d'IDF
dep = ["75","77","78","91","92","93","94","95"]

#Date minimale 
date_min = [2010,1,1]

data = get_RNE_Final(dep=dep, activites=activites,date_min=date_min)

  0%|          | 0/13 [00:00<?, ?it/s]

Récupération depuis le cloud 1/13


  8%|▊         | 1/13 [00:02<00:35,  2.96s/it]

Récupération depuis le cloud 2/13


 15%|█▌        | 2/13 [00:08<00:50,  4.60s/it]

...

Filtrage par département et activités


100%|██████████| 13/13 [00:09<00:00,  1.31it/s]


Récupération depuis 2010-1-1
Récupération de l'historique total


100%|██████████| 13/13 [02:06<00:00,  9.72s/it]


Récupération expérience associé 0
Récupération expérience associé 1
Récupération expérience associé 2
Récupération expérience associé 3
Récupération expérience associé 4
Récupération expérience associé 5
Récupération de l'âge moyen


Exemple des nouvelles colonnes obtenues :

In [24]:
data[["nom","age_physique","associe_1_naissance","siren","age0","age1","mean_age","nb_associe","total_exp1","total_rad1","ape_true1"]].head(10)

,nom,age_physique,associe_1_naissance,siren,age0,age1,mean_age,nb_associe,total_exp1,total_rad1,ape_true1
0,SARAFIAN,1950-06-01,NaT,300553369,61.456537,NaN,61.456537,1,0,0,0
1,non-disp,NaT,1939-10-01,301051827,NaN,71.397673,71.397673,1,0,0,0
2,non-disp,NaT,1973-12-01,302315510,NaN,40.950034,40.950034,1,0,0,0
3,non-disp,NaT,NaT,302452297,NaN,NaN,39.753593,1,0,0,0
4,non-disp,NaT,1964-06-01,303159875,NaN,47.301848,44.551677,2,2,0,0
5,non-disp,NaT,1957-03-01,303968812,NaN,57.240246,47.571983,3,0,0,0
6,non-disp,NaT,1972-04-01,304514698,NaN,41.097878,41.097878,1,0,0,0
7,non-disp,NaT,1952-09-01,305034019,NaN,58.091718,58.091718,1,6,0,0
8,non-disp,NaT,1961-07-01,305147860,NaN,49.117043,44.894365,3,1,0,0
9,non-disp,NaT,1961-06-01,305933681,NaN,49.133470,49.133470,1,1,0,0


In [26]:
MY_BUCKET = "guillaume176"

FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/data_final/idf_10_24.parquet"

with fs.open(FILE_PATH_OUT_S3,"wb") as file_out:
    data.to_parquet(file_out, index=False)


Lien pour le téléchargement pour `idf_10_24`: 

In [31]:
path_data = "guillaume176/diffusion/data_final"
file_list = fs.ls(path_data)[1:]
file_list = ["https://minio.lab.sspcloud.fr/" + f for f in file_list]
print(file_list[0])

https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_final/idf_10_24.parquet
